In [14]:
pip install neurokit2

In [15]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import neurokit2 as nk

**Модель с вычислением вектора признаков**

Загружаем данные

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
exams_path = "/content/drive/MyDrive/exams.csv"
information = pd.read_csv(exams_path)

In [18]:
path_to_signals = '/content/drive/MyDrive/exams_part5.hdf5'
signals_folder = h5py.File(path_to_signals, 'r')

In [19]:
exam_id = np.array(signals_folder['exam_id'])
exam_id = exam_id[0:20000]

In [20]:
id_dict = {}
for i in range (0,345779):
  id_dict[information.exam_id[i]] = i


In [21]:
results = np.eye(20000, 6)

for i in range(0,20000):
  ind = id_dict[exam_id[i]]

  if information['1dAVb'][ind]:
    results[i][0] = 1
  else:
    results[i][0] = 0

  if information['RBBB'][ind]:
    results[i][1] = 1
  else:
    results[i][1] = 0

  if information['LBBB'][ind]:
    results[i][2] = 1
  else:
    results[i][2] = 0

  if information['SB'][ind]:
    results[i][3] = 1
  else:
    results[i][3] = 0

  if information['ST'][ind]:
    results[i][4] = 1
  else:
    results[i][4] = 0

  if information['AF'][ind]:
    results[i][5] = 1
  else:
    results[i][5] = 0


In [22]:
signals = np.array(signals_folder['tracings'][:,:,0])

Выделяем признаки из сигнала

In [23]:
data = []
res = []
for i in range(0,20000):
  features = []
  try:
    signal_peak, waves_peak = nk.ecg_delineate(signals[i,:], sampling_rate = 400)
  except:
    continue
  sig_peaks = {}
  val_peaks = {}

  for key in signal_peak:
    sig_peaks[key] = np.nonzero(signal_peak[key])

  for key in sig_peaks:
    val_peaks[key] = signals[i,:][sig_peaks[key]]

  r_peaks, _ = nk.ecg_peaks(signals[i,:], sampling_rate = 400)
  r_peaks_indices = np.nonzero(r_peaks)[0]
  r_peaks_values = signals[i,:][r_peaks_indices]
  rr_intervals = np.diff(r_peaks_indices)
  features.append(np.mean(rr_intervals))

  P_onsets = np.array(waves_peak['ECG_P_Onsets'][1:-1])
  P_offsets = np.array(waves_peak['ECG_P_Offsets'][1:-1])
  R_onsets = np.array(waves_peak['ECG_R_Onsets'][1:-1])
  R_offsets = np.array(waves_peak['ECG_R_Offsets'][1:-1])
  T_offsets = np.array(waves_peak['ECG_T_Offsets'][1:-1])

  s = R_onsets-P_onsets
  s = s[~np.isnan(s)]
  features.append(np.mean(s))
  s = R_offsets-R_onsets
  s = s[~np.isnan(s)]
  features.append(np.mean(s))
  s = P_offsets-P_onsets
  s = s[~np.isnan(s)]
  features.append(np.mean(s))
  s = T_offsets-R_offsets
  s = s[~np.isnan(s)]
  features.append(np.mean(s))
  data.append(features)
  res.append(results[i])



/usr/local/lib/python3.10/dist-packages/neurokit2/signal/signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
/usr/local/lib/python3.10/dist-packages/neurokit2/signal/signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local

In [24]:
data = np.array(data)

Модель LSTM

In [46]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(64, input_shape=(5,1)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(6, activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.F1Score()], optimizer='Adam')
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 64)                16896     
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 6)                 390       
                                                                 
Total params: 17286 (67.52 KB)
Trainable params: 17286 (67.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(data, results, epochs=20)

Epoch 1/20
157/157 [==============================] - 259s 2s/step - loss: 0.0319 - accuracy: 0.1310 - precision_4: 0.0538 - recall_4: 0.0119 - f1_score: 0.0378
Epoch 2/20
157/157 [==============================] - 262s 2s/step - loss: 0.0237 - accuracy: 0.0840 - precision_4: 0.0769 - recall_4: 0.0017 - f1_score: 0.0353
Epoch 3/20
157/157 [==============================] - 253s 2s/step - loss: 0.0220 - accuracy: 0.0824 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - f1_score: 0.0363
Epoch 4/20
157/157 [==============================] - 257s 2s/step - loss: 0.0208 - accuracy: 0.0918 - precision_4: 0.5000 - recall_4: 0.0017 - f1_score: 0.0356
Epoch 5/20
157/157 [==============================] - 255s 2s/step - loss: 0.0203 - accuracy: 0.0732 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - f1_score: 0.0358
Epoch 6/20
157/157 [==============================] - 257s 2s/step - loss: 0.0200 - accuracy: 0.0742 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - f1_score: 0.0427
Epoch 7/20

In [ ]:
predict = model.predict(data[15000:])

154/154 [==============================] - 2s 5ms/step


In [ ]:
print(predict[200:300])

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0.